In [36]:
import kagglehub
import pytesseract
from PIL import Image
import requests
import pandas as pd
from transformers import AutoProcessor, BlipForConditionalGeneration
import ollama 

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' # my dir where i installed tesseract

# Download latest version
path = kagglehub.dataset_download("akuppps/dankmemes-reddit-top-comments")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Leander Antony\.cache\kagglehub\datasets\akuppps\dankmemes-reddit-top-comments\versions\1


In [37]:
df = pd.read_csv("C:\\Users\\Leander Antony\\.cache\\kagglehub\\datasets\\akuppps\\dankmemes-reddit-top-comments\\versions\\1\\dankmemes_Top_Comments.csv")

In [38]:
df.sample(10)

,Unnamed: 0,title,score,id,url,comms_num,created,body,timestamp
1219,1219,Comment,5,i39ndvp,NaN,0,1.649018e+09,"she, more like shit",2022-04-03 16:41:19
1539,1539,Comment,1,i395g2f,NaN,0,1.649011e+09,I prefer ram ranch,2022-04-03 14:37:24
591,591,Mom is OP,6541,tdra4t,https://i.redd.it/rpiap4ctqan81.jpg,18,1.647241e+09,NaN,2022-03-14 02:51:45
333,333,Literally 1984,13484,t7r6fj,https://i.redd.it/j6lyoqx60pl81.gif,50,1.646542e+09,NaN,2022-03-05 23:41:01
1551,1551,Comment,240,i3954w2,NaN,0,1.649011e+09,Meanwhile germans:,2022-04-03 14:35:12
138,138,Theist gnostic,27965,ts4a3x,https://i.redd.it/l93zwspx9iq81.jpg,58,1.648640e+09,NaN,2022-03-30 07:27:47
1703,1703,Comment,0,i38zw97,NaN,0,1.649009e+09,Hard to compare. While the US acknowledged its...,2022-04-03 13:59:20
1682,1682,Comment,2,i390imu,NaN,0,1.649009e+09,O nyooooooo he is cheating. He place colored p...,2022-04-03 14:03:32
819,819,WHERE ARE THEY,5854,tumhfz,https://i.redd.it/ghp1f14j45r81.gif,39,1.648916e+09,NaN,2022-04-02 12:14:35
1494,1494,Comment,1,i396zlg,NaN,0,1.649012e+09,Ducc 2:1 The power of Ducc could not be held b...,2022-04-03 14:48:18


In [39]:
len(df["title"])

1898

In [40]:
df.isna().sum()

Unnamed: 0      0
title           0
score           0
id              0
url           899
comms_num       0
created         0
body          999
timestamp       0
dtype: int64

In [41]:
df.dropna(subset=["url"], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999 entries, 0 to 998
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  999 non-null    int64  
 1   title       999 non-null    object 
 2   score       999 non-null    int64  
 3   id          999 non-null    object 
 4   url         999 non-null    object 
 5   comms_num   999 non-null    int64  
 6   created     999 non-null    float64
 7   body        0 non-null      object 
 8   timestamp   999 non-null    object 
dtypes: float64(1), int64(3), object(5)
memory usage: 78.0+ KB


In [42]:
urls = df.url

In [43]:
urls[1:9]

1    https://i.redd.it/cmem70sowcm81.gif
2    https://i.redd.it/5kxyntrizhm81.gif
3    https://i.redd.it/p4rc3d0yl1q81.jpg
4    https://i.redd.it/ycsi7qy775q81.gif
5    https://i.redd.it/70uum4rpxll81.gif
6    https://i.redd.it/fepe187jfjn81.gif
7    https://i.redd.it/to10s17e2bp81.gif
8    https://i.redd.it/z3i1fbc6kfq81.gif
Name: url, dtype: object

In [47]:
def extract_text(url):
    image = Image.open(requests.get(url, stream=True).raw)
    text = pytesseract.image_to_string(image)
    return text


def image_to_text(url):
    processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    image = Image.open(requests.get(url, stream=True).raw)
    text = "A picture of"
    
    inputs = processor(images=image, text=text, return_tensors="pt")
    
    outputs = model.generate(**inputs)
    caption = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
    return caption


def generate_caption(extracted_text, text_from_image):
    prompt = f"""you are a humoros meme caption generator. generate one caption for this text extracted from the meme {extracted_text} 
    and this description of the image {text_from_image}' """
    response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']


In [48]:
def pipeline(url):
    extracted_text = extract_text(url)
    text_from_image = image_to_text(url)
    generated_caption = generate_caption(extracted_text, text_from_image)
    return generated_caption

In [ ]:
for i in range(7):
    print(urls[i])
    captions = pipeline(urls[i])
    print(captions)

https://i.redd.it/zysreb6cv3q81.gif
 Caption: "When Will Smith realizes Jada's reaction wasn't about him, but the guy behind him popcorn spilling dramatically!" (with a picture of a man laughing hysterically followed by a confused expression)
https://i.redd.it/cmem70sowcm81.gif
 "When you thought your high school math skills would suffice for rocket science..."
https://i.redd.it/5kxyntrizhm81.gif
 USA: "When you realize Ukraine has all the good stuff under the ground..." *Picks up phone* "Oh hi, Saudi Arabia... I was wondering if you have a minute to talk about oil deposits?"
https://i.redd.it/p4rc3d0yl1q81.jpg
 When someone tells a dad joke about *your* wife, Will Smith's reaction! 😜🚀💥
https://i.redd.it/ycsi7qy775q81.gif
 "Preparing for the World Cup of Wit: When your joke is as round as a soccer ball, but it needs to be funny as an onion!"
https://i.redd.it/70uum4rpxll81.gif
